#### Setup 
- Import required packages and set the file directory

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import pyproj


file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/pilot'

#### Read in species data

- First read in a text file containing the names of all the taxa included in the modelling <br>
- Then create an empty dictionary <br>
- Load in the csv files containing the species occurrence points <br>
- Store them in the dictionary


In [ ]:
#access file with list of taxa names
taxa=pd.read_csv(file_dir+"/data/spec_filtered/taxa.txt",header=None)
taxa.columns=["taxon"]

In [ ]:
species_occ_dict={}

for i in taxa["taxon"]:
    taxon_data = pd.read_csv(file_dir+"/data/spec_filtered/%s.csv"%i)
    #add species dataframe to dict
    species_occ_dict["%s"%i] = taxon_data  
    #check whether all species have been included and inspect dictionary
if len(species_occ_dict.keys())==len(taxa["taxon"]):
    print("All species dataframes now in dictionary")
else:
    print("Error: not all species dataframe included")

#### Plot species occurrence points
- Create a basemap <br>
- Plot the points on the basemap and export it

In [ ]:
%%capture
for key in species_occ_dict:
    data = species_occ_dict[key]
    spec = data["taxon_name"][0]
    spec=spec.replace(" ","_")
    #lat = data["decimal_latitude"]
    #lon = data["decimal_longitude"]
     
    data['coordinates'] = list(zip(data["decimal_longitude"], data["decimal_latitude"]))
    data['coordinates'] = data["coordinates"].apply(Point)
    data["present/pseudo_absent"]=1
    geo_data=geopandas.GeoDataFrame(data, geometry='coordinates',crs={'init' :'epsg:4326'})
    
    lat=data["decimal_latitude"]
    lon=data["decimal_longitude"]
    meanlat=sum(lat)/len(lat)
    meanlon=sum(lon)/len(lon)
       
    #change projection to azimuthal equidistant to be able to plot  points on aeqd map
    geo_data = geo_data.to_crs({'init': 'esri:54032'}) 
    
    fig=plt.figure(figsize=(12, 12))
    map = Basemap(projection="aeqd",resolution="l", ellps="WGS84",
        lat_0=meanlat,lon_0=meanlon,
        width=3000*2500,height=3000*2500,
        suppress_ticks=False,)
    
    map.drawmapboundary(fill_color='white')
    map.fillcontinents(color='lightgray',lake_color='white')
    map.drawcountries(color="gray")
    map.drawcoastlines(linewidth=0.2)

    for i in range(0,len(lon)):
        xpt,ypt = map(lon[i],lat[i])
        map.plot([xpt],[ypt],"ko",markersize=5,color="yellow",markeredgecolor="black")
    speci=spec.replace("_"," ")
    plt.title("Filtered occurrences for %s"%speci,fontsize=20,fontweight="bold")
    plt.savefig(file_dir+'/results/{}/{}_occurrence_map.png'.format(spec,spec),pad_inches=0.0, bbox_inches='tight')
    plt.show()
    plt.close()